<a href="https://colab.research.google.com/github/kurtlee1984/ML_20210602/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from urllib.request import urlretrieve
url = "https://raw.githubusercontent.com/Elwing-Chou/ml0602/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://raw.githubusercontent.com/Elwing-Chou/ml0602/main/poem_test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f6683fcbe90>)

In [8]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
# 分類: 輸出 0 1 2 3
# series.replace(字典)
# {"李白":0, "杜甫":1, "白居易":2}
# series.unique/series.value_counts
u = train["作者"].unique()
name2index = {n:i for i, n in enumerate(u)}
index2name = {i:n for i, n in enumerate(u)}
print(name2index)
print(index2name)
y_train = train["作者"].replace(name2index)
y_test = test["作者"].replace(name2index)
y_test

In [13]:
import jieba
def poemcut(p):
  return " ".join(jieba.cut(p))
x_train = train["內容"].apply(poemcut)
x_test = test["內容"].apply(poemcut)
x_train

0       平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 \r\n 暝 色入 高樓 ， 有人 樓上 ...
1       青天 有 月 來 幾時 ， 我今 停杯 一問 之 ： 人攀 明月 不可 得 ， 月行 卻 與...
2       燕草 如碧絲 ， 秦桑低 綠枝 。 當君 懷歸日 ， 是 妾 斷腸時 。 春風 不 相識 ，...
3       誰 家玉笛 暗飛聲 ， 散 入春 風滿 洛城 。 此 夜曲 中聞折 柳 ， 何人 不起 故園情 。
4       西上 蓮 花山 ， 迢迢 見 明星 。 ( 西 上   一作 ： 西 嶽 ) 素 手把 芙蓉...
                              ...                        
2726    零落 桐葉雨 ， 蕭條 槿 花風 。 悠悠 早 秋意 ， 生此 幽閒 中 。 況 與 故人 ...
2727    離離 暑雲散 ， 嫋嫋 涼風起 。 池上 秋 又 來 ， 荷花 半成 子 。 朱顏易銷 歇 ...
2728    涼風 冷露 蕭索天 ， 黃 蒿 紫菊荒 涼田 。 繞 塚 秋花少 顏色 ， 細蟲 小蝶飛 翻...
2729    往歲 曾 為 西邑吏 ， 慣 從 駱口 到 南秦 。 \r\n 三時 雲冷多 飛雪 ， 二月...
2730    笛 竹出 蘄 春 ， 霜刀 劈翠筠 。 織成 雙鎖 簟 ， 寄 與 獨眠 人 。 卷作筒 中...
Name: 內容, Length: 2731, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(x_train)
len(vec.vocabulary_)

52294

In [15]:
#稀疏矩陣
x_train_count = vec.transform(x_train)
x_test_count = vec.transform(x_test)
x_test_count

<30x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 352 stored elements in Compressed Sparse Row format>